In [93]:
import pandas as pd

In [94]:
import numpy as np

### Carregar dados

In [95]:
jogos = pd.read_csv("data/brasileiro-a.csv")

In [96]:
jogos.head()

,campeonato,ano,rodada,data,mandante,visitante,placar
0,brasileiro-a,2018,6,21.05. 20:00,Internacional,Chapecoense,3 : 0
1,brasileiro-a,2018,6,20.05. 19:00,Fluminense,Atlético-PR,2 : 0
2,brasileiro-a,2018,6,20.05. 16:00,América-MG,Botafogo,1 : 0
3,brasileiro-a,2018,6,20.05. 16:00,Paraná,Grêmio,0 : 0
4,brasileiro-a,2018,6,20.05. 16:00,São Paulo,Santos,1 : 0


#### A query abaixo possibilita restringir o espaço amostral

In [97]:
#jogos.query('ano == 2017 & rodada > 25', inplace=True)

In [98]:
jogos.head()

,campeonato,ano,rodada,data,mandante,visitante,placar
0,brasileiro-a,2018,6,21.05. 20:00,Internacional,Chapecoense,3 : 0
1,brasileiro-a,2018,6,20.05. 19:00,Fluminense,Atlético-PR,2 : 0
2,brasileiro-a,2018,6,20.05. 16:00,América-MG,Botafogo,1 : 0
3,brasileiro-a,2018,6,20.05. 16:00,Paraná,Grêmio,0 : 0
4,brasileiro-a,2018,6,20.05. 16:00,São Paulo,Santos,1 : 0


In [99]:
jogos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6185 entries, 0 to 6184
Data columns (total 7 columns):
campeonato    6185 non-null object
ano           6185 non-null int64
rodada        6185 non-null int64
data          6185 non-null object
mandante      6185 non-null object
visitante     6185 non-null object
placar        6185 non-null object
dtypes: int64(2), object(5)
memory usage: 338.3+ KB


In [100]:
jogos_obj = jogos.select_dtypes(['object'])
jogos[jogos_obj.columns] = jogos_obj.apply(lambda x: x.str.strip())

### Lista de times participantes

In [101]:
times = list(jogos["mandante"].unique())

In [102]:
times.sort()

In [103]:
times

['América-MG',
 'América-RN',
 'Atlético-GO',
 'Atlético-MG',
 'Atlético-PR',
 'Avaí',
 'Bahia',
 'Barueri',
 'Botafogo',
 'Brasiliense',
 'Ceará',
 'Chapecoense',
 'Corinthians',
 'Coritiba',
 'Criciúma',
 'Cruzeiro',
 'Figueirense',
 'Flamengo',
 'Fluminense',
 'Fortaleza',
 'Goias',
 'Grêmio',
 'Guarani',
 'Internacional',
 'Ipatinga',
 'Joinville',
 'Juventude',
 'Náutico',
 'Palmeiras',
 'Paraná',
 'Paysandu',
 'Ponte Preta',
 'Portuguesa',
 'Santa Cruz',
 'Santo Andre',
 'Santos',
 'Sao Caetano',
 'Sport Recife',
 'São Paulo',
 'Vasco',
 'Vitória']

In [104]:
len(times)

41

### Transformar placar em 2 colunas

In [105]:
def tratarPlacar(row, mandante):
    placar = row["placar"].split(" : ")
    if (mandante):
        return int(placar[0])
    return int(placar[1])

In [106]:
jogos['gols_mandante'] = jogos.apply (lambda row: tratarPlacar(row, True),axis=1)

In [107]:
jogos['gols_visitante'] = jogos.apply (lambda row: tratarPlacar(row, False),axis=1)

In [108]:
del jogos["placar"]

In [109]:
jogos.describe()

,ano,rodada,gols_mandante,gols_visitante
count,6185.000000,6185.000000,6185.000000,6185.000000
mean,2009.648504,20.210833,1.600970,1.048181
std,4.515351,11.671752,1.253146,1.041168
min,2003.000000,1.000000,0.000000,0.000000
25%,2005.000000,10.000000,1.000000,0.000000
50%,2010.000000,20.000000,1.000000,1.000000
75%,2014.000000,30.000000,2.000000,2.000000
max,2018.000000,46.000000,7.000000,7.000000


### Tratar coluna de data

In [110]:
def tratarData(row):
    data = row["data"].split(".")    
    return data[0] + "-" + data[1]

In [111]:
jogos['data'] = jogos.apply (lambda row: tratarData(row),axis=1)

In [112]:
jogos.head(2)

,campeonato,ano,rodada,data,mandante,visitante,gols_mandante,gols_visitante
0,brasileiro-a,2018,6,21-05,Internacional,Chapecoense,3,0
1,brasileiro-a,2018,6,20-05,Fluminense,Atlético-PR,2,0


### Criar coluna com resultado

In [113]:
def resultado(row):
    if (row["gols_mandante"] > row["gols_visitante"]):
        return "MANDANTE"
    elif (row["gols_mandante"] < row["gols_visitante"]):
        return "VISITANTE"
    else:
        return "EMPATE"

In [114]:
jogos['resultado'] = jogos.apply (lambda row: resultado(row),axis=1)

In [115]:
jogos.head()

,campeonato,ano,rodada,data,mandante,visitante,gols_mandante,gols_visitante,resultado
0,brasileiro-a,2018,6,21-05,Internacional,Chapecoense,3,0,MANDANTE
1,brasileiro-a,2018,6,20-05,Fluminense,Atlético-PR,2,0,MANDANTE
2,brasileiro-a,2018,6,20-05,América-MG,Botafogo,1,0,MANDANTE
3,brasileiro-a,2018,6,20-05,Paraná,Grêmio,0,0,EMPATE
4,brasileiro-a,2018,6,20-05,São Paulo,Santos,1,0,MANDANTE


In [116]:
jogos.to_csv("data/jogos-resultado.csv", sep=",")

### Totais de resultados por ano

In [117]:
jogos.groupby(["ano", "resultado"])[["ano"]].count()

ano
ano  resultado     
2003 EMPATE     142
     MANDANTE   297
     VISITANTE  113
2004 EMPATE     140
     MANDANTE   288
     VISITANTE  124
2005 EMPATE     103
     MANDANTE   235
     VISITANTE  124
2006 EMPATE      97
     MANDANTE   191
     VISITANTE   92
2007 EMPATE      90
     MANDANTE   192
     VISITANTE   98
2008 EMPATE      96
     MANDANTE   208
     VISITANTE   76
2009 EMPATE     102
     MANDANTE   195
     VISITANTE   83
2010 EMPATE     118
     MANDANTE   179
     VISITANTE   83
2011 EMPATE     105
     MANDANTE   184
     VISITANTE   91
2012 EMPATE     105
     MANDANTE   183
     VISITANTE   92
2013 EMPATE     108
     MANDANTE   184
     VISITANTE   88
2014 EMPATE      92
     MANDANTE   197
     VISITANTE   91
2015 EMPATE      91
     MANDANTE   200
     VISITANTE   89
2016 EMPATE      95
     MANDANTE   202
     VISITANTE   83
2017 EMPATE     103
     MANDANTE   167
     VISITANTE  110
2018 EMPATE      19
     MANDANTE    32
     VISITANTE    8

### Totais por resultado

In [118]:
totais_resultados = jogos.groupby("resultado").size()

In [119]:
total_jogos = totais_resultados.sum()

In [120]:
resultados_DF = pd.DataFrame({'resultados': totais_resultados.keys(), 'totais': totais_resultados.values})

In [121]:
resultados_DF

,resultados,totais
0,EMPATE,1606
1,MANDANTE,3134
2,VISITANTE,1445


In [122]:
def percentual(valor, total):
    return round(valor/total, 4)

In [123]:
resultados_DF['%'] = resultados_DF.apply (lambda row: percentual(row["totais"], total_jogos),axis=1)

In [124]:
resultados_DF

,resultados,totais,%
0,EMPATE,1606,0.2597
1,MANDANTE,3134,0.5067
2,VISITANTE,1445,0.2336


### Calcular resultados de um time

In [125]:
def resultadosTime(time, casa, resultado):
    return len(jogos[(jogos[casa] == time) & (jogos["resultado"] == resultado)]) 

In [126]:
resultados_time_DF = pd.DataFrame({'time': times })

In [127]:
resultados_time_DF["vitorias_casa"] = resultados_time_DF.apply (lambda row: resultadosTime(row["time"], "mandante", "MANDANTE"),axis=1)
resultados_time_DF["empates_casa"] = resultados_time_DF.apply (lambda row: resultadosTime(row["time"], "mandante", "EMPATE"),axis=1)
resultados_time_DF["derrotas_casa"] = resultados_time_DF.apply (lambda row: resultadosTime(row["time"], "mandante", "VISITANTE"),axis=1)
resultados_time_DF["vitorias_fora"] = resultados_time_DF.apply (lambda row: resultadosTime(row["time"], "visitante", "VISITANTE"),axis=1)
resultados_time_DF["empates_fora"] = resultados_time_DF.apply (lambda row: resultadosTime(row["time"], "visitante", "EMPATE"),axis=1)
resultados_time_DF["derrotas_fora"] = resultados_time_DF.apply (lambda row: resultadosTime(row["time"], "visitante", "MANDANTE"),axis=1)

In [128]:
resultados_time_DF.head(3)

,time,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora
0,América-MG,17,10,14,1,11,29
1,América-RN,2,3,14,2,2,15
2,Atlético-GO,25,17,34,14,22,40


In [129]:
resultados_time_DF.eval("jogos_casa = vitorias_casa + empates_casa + derrotas_casa", inplace=True)
resultados_time_DF.eval("jogos_fora = vitorias_fora + empates_fora + derrotas_fora", inplace=True)
resultados_time_DF.eval("jogos_total = jogos_fora + jogos_fora", inplace=True)

In [130]:
resultados_time_DF.head(3)

,time,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora,jogos_casa,jogos_fora,jogos_total
0,América-MG,17,10,14,1,11,29,41,41,82
1,América-RN,2,3,14,2,2,15,19,19,38
2,Atlético-GO,25,17,34,14,22,40,76,76,152


In [131]:
def aproveitamentoMandante(row):
    total_jogos_mandante = row["vitorias_casa"] + row["empates_casa"] + row["derrotas_casa"]
    return (3*row["vitorias_casa"] + row["empates_casa"]) / (3*total_jogos_mandante)

In [132]:
resultados_time_DF["aprov_casa"] = resultados_time_DF.apply (lambda row: aproveitamentoMandante(row),axis=1)
resultados_time_DF.head(3)


,time,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora,jogos_casa,jogos_fora,jogos_total,aprov_casa
0,América-MG,17,10,14,1,11,29,41,41,82,0.495935
1,América-RN,2,3,14,2,2,15,19,19,38,0.157895
2,Atlético-GO,25,17,34,14,22,40,76,76,152,0.403509


In [133]:
def aproveitamentoVisitante(row):
    total_jogos_visitante = row["vitorias_fora"] + row["empates_fora"] + row["derrotas_fora"]
    return (3*row["vitorias_fora"] + row["empates_fora"]) / (3*total_jogos_visitante)

In [134]:
resultados_time_DF["aprov_fora"] = resultados_time_DF.apply (lambda row: aproveitamentoVisitante(row),axis=1)
resultados_time_DF.head(3)

,time,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora,jogos_casa,jogos_fora,jogos_total,aprov_casa,aprov_fora
0,América-MG,17,10,14,1,11,29,41,41,82,0.495935,0.113821
1,América-RN,2,3,14,2,2,15,19,19,38,0.157895,0.140351
2,Atlético-GO,25,17,34,14,22,40,76,76,152,0.403509,0.280702


In [135]:
def aproveitamentoTotal(row):
    total_jogos_mandante = row["vitorias_casa"] + row["empates_casa"] + row["derrotas_casa"]
    total_jogos_visitante = row["vitorias_fora"] + row["empates_fora"] + row["derrotas_fora"]
    return (3*row["vitorias_fora"] + row["empates_fora"] + 3*row["vitorias_casa"] + row["empates_casa"]) / (3*total_jogos_mandante + 3*total_jogos_visitante)

In [136]:
resultados_time_DF["aprov_total"] = resultados_time_DF.apply (lambda row: aproveitamentoTotal(row),axis=1)
resultados_time_DF.sort_values(["aprov_total"], ascending=[0]).head(3)

,time,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora,jogos_casa,jogos_fora,jogos_total,aprov_casa,aprov_fora,aprov_total
38,São Paulo,178,73,47,100,80,118,298,298,596,0.678971,0.425056,0.552013
12,Corinthians,158,69,52,88,88,103,279,279,558,0.648746,0.420550,0.534648
15,Cruzeiro,170,67,61,104,59,135,298,298,596,0.645414,0.414989,0.530201


In [137]:
resultados_time_DF = resultados_time_DF[["time", "jogos_casa", "jogos_fora", "jogos_total", "aprov_casa", "aprov_fora", "aprov_total"]].sort_values(["aprov_total"], ascending=[0])

In [138]:
def golsMandante(time):
    return jogos[jogos["mandante"] == time][["gols_mandante"]].sum()

In [139]:
def golsVisitante(time):
    return jogos[jogos["visitante"] == time]["gols_visitante"].sum()

In [140]:
resultados_time_DF["gols_mandante"] = resultados_time_DF.apply (lambda row: golsMandante(row["time"]),axis=1)
resultados_time_DF["gols_visitante"] = resultados_time_DF.apply (lambda row: golsVisitante(row["time"]),axis=1)

resultados_time_DF.eval("total_gols = gols_mandante + gols_visitante", inplace=True)
resultados_time_DF.eval("media_gols_casa = gols_mandante/jogos_casa", inplace=True)
resultados_time_DF.eval("media_gols_fora = gols_visitante/jogos_fora", inplace=True)
resultados_time_DF.eval("media_gols = (gols_mandante + gols_visitante)/jogos_total", inplace=True)

In [141]:
resultados_time_DF.head()


,time,jogos_casa,jogos_fora,jogos_total,aprov_casa,aprov_fora,aprov_total,gols_mandante,gols_visitante,total_gols,media_gols_casa,media_gols_fora,media_gols
38,São Paulo,298,298,596,0.678971,0.425056,0.552013,533,367,900,1.788591,1.231544,1.510067
12,Corinthians,279,279,558,0.648746,0.420550,0.534648,441,315,756,1.580645,1.129032,1.354839
15,Cruzeiro,298,298,596,0.645414,0.414989,0.530201,555,365,920,1.862416,1.224832,1.543624
35,Santos,297,298,596,0.675645,0.361298,0.518207,551,357,908,1.855219,1.197987,1.523490
23,Internacional,279,279,558,0.663082,0.358423,0.510753,447,296,743,1.602151,1.060932,1.331541


In [142]:
resultados_time_DF = resultados_time_DF[["time", "jogos_total", "aprov_casa", "aprov_fora", "aprov_total", "media_gols_casa", "media_gols_fora", "media_gols"]]

In [143]:
resultados_time_DF.set_index("time", inplace=True)

In [144]:
resultados_time_DF.sort_values(["aprov_total"], ascending = [0]).head(3)

,jogos_total,aprov_casa,aprov_fora,aprov_total,media_gols_casa,media_gols_fora,media_gols
time,,,,,,,
São Paulo,596,0.678971,0.425056,0.552013,1.788591,1.231544,1.510067
Corinthians,558,0.648746,0.420550,0.534648,1.580645,1.129032,1.354839
Cruzeiro,596,0.645414,0.414989,0.530201,1.862416,1.224832,1.543624


In [145]:
resultados_time_DF_DESC = resultados_time_DF.describe(percentiles=[0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1])

In [146]:
resultados_time_DF_DESC

,jogos_total,aprov_casa,aprov_fora,aprov_total,media_gols_casa,media_gols_fora,media_gols
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,301.707317,0.548050,0.269632,0.408840,1.502861,0.961953,1.232410
std,208.841357,0.095735,0.089082,0.083228,0.245214,0.187350,0.191986
min,38.000000,0.157895,0.087719,0.149123,0.631579,0.368421,0.631579
0%,38.000000,0.157895,0.087719,0.149123,0.631579,0.368421,0.631579
10%,42.000000,0.438596,0.140351,0.307018,1.264463,0.756098,1.048780
20%,82.000000,0.500000,0.198413,0.350877,1.315789,0.870968,1.131579
30%,134.000000,0.520325,0.210526,0.375661,1.408840,0.894737,1.166667
40%,172.000000,0.538813,0.265193,0.383085,1.466387,0.939560,1.210046
50%,216.000000,0.555556,0.270308,0.415054,1.556017,1.000000,1.222689


In [147]:
percentis = resultados_time_DF_DESC.loc[["10%","20%","30%","40%","50%","60%","70%","80%","90%"]]

In [148]:
percentis

,jogos_total,aprov_casa,aprov_fora,aprov_total,media_gols_casa,media_gols_fora,media_gols
10%,42.0,0.438596,0.140351,0.307018,1.264463,0.756098,1.048780
20%,82.0,0.500000,0.198413,0.350877,1.315789,0.870968,1.131579
30%,134.0,0.520325,0.210526,0.375661,1.408840,0.894737,1.166667
40%,172.0,0.538813,0.265193,0.383085,1.466387,0.939560,1.210046
50%,216.0,0.555556,0.270308,0.415054,1.556017,1.000000,1.222689
60%,364.0,0.580729,0.290123,0.430672,1.578947,1.010830,1.289474
70%,480.0,0.593548,0.332031,0.456380,1.611111,1.060932,1.333333
80%,558.0,0.610514,0.358423,0.482103,1.702509,1.104167,1.375000
90%,596.0,0.653524,0.375000,0.510753,1.776256,1.164062,1.440299


In [149]:
def get_stars(array,value):
    print ()
    idx = (np.abs(array-value)).argmin()
    near = array[idx][0]
    index = np.where(array==near)[0][0]
    
    if (value > near):
        index = index + 1  
       
    return index + 1

In [150]:
stars_DF = resultados_time_DF.copy()

In [151]:
stars_DF["aprov_total"] = resultados_time_DF.apply (lambda row: get_stars(percentis[["aprov_total"]].values, row["aprov_total"]),axis=1)

In [152]:
stars_DF["aprov_casa"] = resultados_time_DF.apply (lambda row: get_stars(percentis[["aprov_casa"]].values, row["aprov_casa"]),axis=1)

In [153]:
stars_DF["aprov_fora"] = resultados_time_DF.apply (lambda row: get_stars(percentis[["aprov_fora"]].values, row["aprov_fora"]),axis=1)

In [154]:
stars_DF["media_gols_casa"] = resultados_time_DF.apply (lambda row: get_stars(percentis[["media_gols_casa"]].values, row["media_gols_casa"]),axis=1)

In [155]:
stars_DF["media_gols_fora"] = resultados_time_DF.apply (lambda row: get_stars(percentis[["media_gols_fora"]].values, row["media_gols_fora"]),axis=1)

In [156]:
stars_DF["media_gols"] = resultados_time_DF.apply (lambda row: get_stars(percentis[["media_gols"]].values, row["media_gols"]),axis=1)

In [157]:
stars_DF.eval("forca_casa = (3*aprov_casa + media_gols_casa)/4", inplace=True)

In [158]:
stars_DF.eval("forca_fora = (3*aprov_fora + media_gols_fora)/4", inplace=True)

In [159]:
stars_DF.eval("forca = (3*aprov_total + 1*media_gols)/4", inplace=True)

In [160]:
stars_DF.sort_values(["forca"], ascending=[0])

,jogos_total,aprov_casa,aprov_fora,aprov_total,media_gols_casa,media_gols_fora,media_gols,forca_casa,forca_fora,forca
time,,,,,,,,,,
São Paulo,596,10,10,10,10,10,10,10.00,10.00,10.00
Santos,596,10,9,10,10,10,10,10.00,9.25,10.00
Cruzeiro,596,9,10,10,10,10,10,9.25,10.00,10.00
Corinthians,558,9,10,10,7,9,8,8.50,9.75,9.50
Atlético-MG,558,8,8,9,9,9,10,8.25,8.25,9.25
Palmeiras,512,9,9,9,8,9,9,8.75,9.00,9.00
Grêmio,554,10,8,9,9,6,8,9.75,7.50,8.75
Internacional,558,10,8,9,7,7,7,9.25,7.75,8.50
Fluminense,596,6,10,8,6,10,9,6.00,10.00,8.25


In [161]:
stars_DF.to_csv(path_or_buf="stars.csv",sep=";")